In [27]:
import random

def generate_trials(seed=None):
    """
    Generates a list of trials for a multisensory experiment with non-uniform transitional probabilities.
    
    In the visual modality, there are two predictive stimuli (vertical, horizontal) and one neutral stimulus (neutralV).
    The probabilities are as follows (with weights shown in parentheses):
    
      - Under one mapping (Condition A):
          * Horizontal: predicted outcome is 'CW' (weight 3) and unpredicted outcome is 'CCW' (weight 1)
          * Vertical:   predicted outcome is 'CCW' (weight 3) and unpredicted outcome is 'CW' (weight 1)
      - Under the alternate mapping (Condition B):
          * Horizontal: predicted outcome is 'CCW' (weight 3) and unpredicted outcome is 'CW' (weight 1)
          * Vertical:   predicted outcome is 'CW' (weight 3) and unpredicted outcome is 'CCW' (weight 1)
    
    The neutral condition is always balanced:
      - neutralV -> 'CW' (weight 1) and neutralV -> 'CCW' (weight 1).
    
    The auditory modality is structured analogously:
    
      - Two predictive tones (1000Hz, 1600Hz) and one neutral tone (neutralA).
      - Under Condition A (Auditory):
          * 1000Hz: predicted outcome is '100Hz' (weight 3) and unpredicted outcome is '160Hz' (weight 1)
          * 1600Hz: predicted outcome is '160Hz' (weight 3) and unpredicted outcome is '100Hz' (weight 1)
      - Under Condition B (Auditory):
          * 1000Hz: predicted outcome is '160Hz' (weight 3) and unpredicted outcome is '100Hz' (weight 1)
          * 1600Hz: predicted outcome is '100Hz' (weight 3) and unpredicted outcome is '160Hz' (weight 1)
      - The neutral condition is balanced:
          * neutralA -> '100Hz' (weight 1) and neutralA -> '160Hz' (weight 1)
    
    Trials are generated by taking the full cross–product of visual and auditory pairs.
    Each unique visual–auditory combination is repeated a number of times equal to the product
    of the visual weight and the auditory weight. Thus, the total number of trials is:
    
         Total visual weight (10) × Total auditory weight (10) = 100 trials.
    
    Parameters:
      seed (int, optional): Seed for the random number generator.
    
    Returns:
      List[dict]: A list of trial dictionaries. Each dictionary contains:
         - 'visual_lead', 'visual_target', 'visual_condition'
         - 'auditory_lead', 'auditory_target', 'auditory_condition'
    """
    if seed is not None:
        random.seed(seed)
    
    # Randomize visual mapping: choose between two conditions.
    if random.choice([True, False]):
        # Condition A: horizontal->CW high, vertical->CCW high.
        visual_pairs = [
            # Vertical stimulus: high probability for CCW, low for CW.
            ('vertical', 'CCW', 'pred', 3),
            ('vertical', 'CW',  'unpred', 1),
            # Horizontal stimulus: high probability for CW, low for CCW.
            ('horizontal', 'CW', 'pred', 3),
            ('horizontal', 'CCW', 'unpred', 1),
        ]
    else:
        # Condition B: horizontal->CCW high, vertical->CW high.
        visual_pairs = [
            # Vertical stimulus: high probability for CW, low for CCW.
            ('vertical', 'CW', 'pred', 3),
            ('vertical', 'CCW', 'unpred', 1),
            # Horizontal stimulus: high probability for CCW, low for CW.
            ('horizontal', 'CCW', 'pred', 3),
            ('horizontal', 'CW', 'unpred', 1),
        ]
    # Add balanced neutral visual pairs.
    visual_pairs.append(('neutralV', 'CW', 'neutral', 2))
    visual_pairs.append(('neutralV', 'CCW', 'neutral', 2))
    
    # Randomize auditory mapping: choose between two conditions.
    if random.choice([True, False]):
        # Condition A: 1000Hz->100Hz high, 1600Hz->160Hz high.
        auditory_pairs = [
            ('1000Hz', '100Hz', 'pred', 3),
            ('1000Hz', '160Hz', 'unpred', 1),
            ('1600Hz', '160Hz', 'pred', 3),
            ('1600Hz', '100Hz', 'unpred', 1),
        ]
    else:
        # Condition B: 1000Hz->160Hz high, 1600Hz->100Hz high.
        auditory_pairs = [
            ('1000Hz', '160Hz', 'pred', 3),
            ('1000Hz', '100Hz', 'unpred', 1),
            ('1600Hz', '100Hz', 'pred', 3),
            ('1600Hz', '160Hz', 'unpred', 1),
        ]
    # Add balanced neutral auditory pairs.
    auditory_pairs.append(('neutralA', '100Hz', 'neutral', 2))
    auditory_pairs.append(('neutralA', '160Hz', 'neutral', 2))
    
    # Build the cross–product.
    trials = []
    for v_lead, v_target, v_cond, v_weight in visual_pairs:
        for a_lead, a_target, a_cond, a_weight in auditory_pairs:
            count = v_weight * a_weight  # number of repetitions for this combination
            for _ in range(count):
                trial = {
                    'visual_lead': v_lead,
                    'visual_target': v_target,
                    'visual_condition': v_cond,
                    'auditory_lead': a_lead,
                    'auditory_target': a_target,
                    'auditory_condition': a_cond
                }
                trials.append(trial)
    
    # Shuffle the trial order to randomize sequence.
    random.shuffle(trials)
    return trials

# Example usage:
if __name__ == "__main__":
    trial_list = generate_trials(seed=42)
    # Print the first 5 trials for inspection.
    for trial in trial_list[:5]:
        print(trial)


{'visual_lead': 'neutralV', 'visual_target': 'CCW', 'visual_condition': 'neutral', 'auditory_lead': 'neutralA', 'auditory_target': '100Hz', 'auditory_condition': 'neutral'}
{'visual_lead': 'vertical', 'visual_target': 'CCW', 'visual_condition': 'pred', 'auditory_lead': '1600Hz', 'auditory_target': '160Hz', 'auditory_condition': 'pred'}
{'visual_lead': 'horizontal', 'visual_target': 'CW', 'visual_condition': 'pred', 'auditory_lead': 'neutralA', 'auditory_target': '160Hz', 'auditory_condition': 'neutral'}
{'visual_lead': 'horizontal', 'visual_target': 'CW', 'visual_condition': 'pred', 'auditory_lead': '1000Hz', 'auditory_target': '100Hz', 'auditory_condition': 'pred'}
{'visual_lead': 'horizontal', 'visual_target': 'CW', 'visual_condition': 'pred', 'auditory_lead': 'neutralA', 'auditory_target': '100Hz', 'auditory_condition': 'neutral'}


In [28]:
import pandas as pd
df = pd.DataFrame(trial_list)

In [32]:
df.groupby(["visual_lead"])["visual_target"].value_counts()

visual_lead  visual_target
horizontal   CW               36
             CCW              12
neutralV     CCW              24
             CW               24
vertical     CCW              36
             CW               12
Name: count, dtype: int64

In [34]:
48*3

144

In [30]:
df.groupby(['visual_lead', 'visual_target', 'auditory_lead', 'auditory_target']).size()

visual_lead  visual_target  auditory_lead  auditory_target
horizontal   CCW            1000Hz         100Hz              3
                                           160Hz              1
                            1600Hz         100Hz              1
                                           160Hz              3
                            neutralA       100Hz              2
                                           160Hz              2
             CW             1000Hz         100Hz              9
                                           160Hz              3
                            1600Hz         100Hz              3
                                           160Hz              9
                            neutralA       100Hz              6
                                           160Hz              6
neutralV     CCW            1000Hz         100Hz              6
                                           160Hz              2
                            1600Hz         10